In [ ]:
# dados_gyroai_generator.py

import numpy as np
import pandas as pd

# -----------------------------
# 1. Criando a linha do tempo
# -----------------------------
# Simulando 1000 pontos de tempo de 0 a 100 segundos
t = np.linspace(0, 100, 1000)

# -----------------------------
# 2. Criando os dados simulados
# -----------------------------
# Simulando movimentos de rotação em três eixos
omega_x = np.sin(t / 10)  # Eixo X
omega_y = np.cos(t / 15)  # Eixo Y
omega_z = np.sin(t / 20)  # Eixo Z

# -----------------------------
# 3. Criando os quaternions (orientação)
# -----------------------------
# theta é o ângulo de rotação
theta = t / 10
# Eixo de rotação normalizado
x, y, z = 1/np.sqrt(3), 1/np.sqrt(3), 1/np.sqrt(3)

# Fórmulas padrão de quaternions
q0 = np.cos(theta / 2)
q1 = np.sin(theta / 2) * x
q2 = np.sin(theta / 2) * y
q3 = np.sin(theta / 2) * z

# -----------------------------
# 4. Gerando ruído realista (como sensores MEMS)
# -----------------------------
np.random.seed(42)  # Garante reprodutibilidade

# Ruído para ω (giroscópio) – ±0.05 rad/s
ruido_gyro = np.random.normal(0.0, 0.05, size=(len(t), 3))
# Ruído para quaternions – ±0.01 (muito sutil)
ruido_quat = np.random.normal(0.0, 0.01, size=(len(t), 4))

# -----------------------------
# 5. Adicionando o ruído aos dados
# -----------------------------
omega_x += ruido_gyro[:, 0]
omega_y += ruido_gyro[:, 1]
omega_z += ruido_gyro[:, 2]

q0 += ruido_quat[:, 0]
q1 += ruido_quat[:, 1]
q2 += ruido_quat[:, 2]
q3 += ruido_quat[:, 3]

# -----------------------------
# 6. Normalizando os quaternions (norma = 1)
# -----------------------------
norma = np.sqrt(q0**2 + q1**2 + q2**2 + q3**2)
q0 /= norma
q1 /= norma
q2 /= norma
q3 /= norma

# -----------------------------
# 7. Criando rótulos de gimbal lock
# -----------------------------
# 5% dos dados terão o label "gimbal_lock"
labels = np.random.choice(["normal", "gimbal_lock"], size=len(t), p=[0.95, 0.05])

# -----------------------------
# 8. Criando o DataFrame final
# -----------------------------
df = pd.DataFrame({
    "tempo": t,
    "omega_x": omega_x,
    "omega_y": omega_y,
    "omega_z": omega_z,
    "q0": q0,
    "q1": q1,
    "q2": q2,
    "q3": q3,
    "label": labels
})

# -----------------------------
# 9. Salvando o CSV
# -----------------------------
caminho = "dados_hibridos_gyroai.csv"
df.to_csv(caminho, index=False)

print(f"✅ Arquivo salvo com sucesso: {caminho}")
print(df.head(10))  # Exibe os 10 primeiros registros
